## 1. Finding Lanes

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('test_image.jpg')


In [2]:
cv2.imshow('result',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#Creating a copy of image and coverting the image to grayscale for better analysis

lane_image = np.copy(image)
gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

# we are using Gaussian Blur to blur the image so that the noise gets reduced
blur = cv2.GaussianBlur(gray,(5,5),0)

cv2.imshow('result',blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Now we will be applying Canny method to identify the edges

An edge corresponds to a region in an image where there is a sharp change in the intensity or a sharp change in colour between adjacent pixels in the image.
The change in brightness over  series of pixels is the Gradient.

A strong Gradient indicates a steep change wehereas a small gradient indicates a shallow change

An image is composed pf pixels and can be therefore be read as a matrix an array of pixel intensities.

We can also represent an image in 2-D coordinate space where X-axis traversesthe images width (#coloumns) and Y-axis traverses images height (#rows)

The canny function will perform a derivateive on our function in both x and y directions thereby measuring the change in intensity with respect to adjacent pixels. It computes the gradient in all directions of our blurred image and is then going to trace our strongest gradients as a series of white pixels.

In [4]:
canny_image = cv2.Canny(blur,50,50)

cv2.imshow('result',canny_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Region of Interest

We are going to specify a region of interest in our image that we are going to use to detect landmines.We will isolate the region of interest and mask everything else.

In [5]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(200,height),(1100,height),(550,250)]]) #creating triangle or polygon retracing the path from our image
    mask = np.zeros_like(image)
    cv2.fillPoly(mask,polygons,255)
    masked_image = cv2.bitwise_and(image,mask) #we are doing bitwise AND between two image matrices of canny image and triangle image tht we created 
    return masked_image

cropped_image = region_of_interest(canny_image)
cv2.imshow('result',cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

In [7]:
def make_coordinates(image,line_parameters):
    

    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array((x1,y1,x2,y2))

In [8]:
def average_slope_intercept(image,lines):
    left_fit = []
    right_fit = []
    
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
            
    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
       # print(left_fit_average, 'left')
        left_line = make_coordinates(image, left_fit_average)
    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        #print(right_fit_average, 'right')
        right_line = make_coordinates(image, right_fit_average)

    return np.array([left_line,right_line])


## Line Detection - Hough  Transform

A technique that will detect stright lines in an image and hence identify the lane line. This technique is known as the Hough Transform.

There are multiple equations with which we can represent a line. Two of them are 

1. y = mx +b
2. xcos(o)+ysin(o) = p

    * p is the perpendicular distance from the origin to the line.
    * o is the angle formed by this perpendicular line and horizontal axis.
    
## Intuition :

* In an image space, a line is plotted as x vs y or plotted as y = mx+b or xcos(o)+ysin(o) = p
* In parameter space (hough), a line is represented by a point "m vs b" or "o vs p"
* Each line is represented as a single point with (m,b) coordinates or (p,o) parameters.

### If we have a point in our "x vs y" space and we have a family of lines that pass through that point which will be having different values for m and b, in hough space (m vs b) if we plot all the values of m and b for all the lines passing through our point, we will get a stright line.

We cannot represent a vertical line in hough space as the value of slope will be infinity.
Therefor, we will represent pur line/point in Polar Coordinate System in terms of o and p (row snd theta) instead of Carteian System.

### xcos(o)+ysin(o) = p


For each line passing through our point in image space with different values of o and p,If we plot O vs p in hough space we will be getting a sinosoidal curve.

A line can be detected by finding the number of intersections between curves. 
The more curves intersecting means the line represented by that intersection passes/crosses more points.

After plotting curves we will divide our hough space in grid system and find a box with majority vote .i.e in which the maximum number of intersections are occuring. We will get a value  of p and o and that line represented using that values will be the line of best fit.

Therefore, we are trying to find which line best fits our data or best describes pur points.

So here we will try to find a line that best defines the edge points in our gradient image.

GRID = Acumulator Array
We will get more precision if we have more no of bins in our grid.

THRESHOLD = Minimum no of intersections/votes needed to accept a candidate line

In [9]:
lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)

averaged_lines = average_slope_intercept(lane_image,lines)
line_image = display_lines(lane_image,averaged_lines)

combo_image = cv2.addWeighted(lane_image,0.8,line_image,1,1)

cv2.imshow('result',combo_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Finding Lanes in a Video

In [10]:
cap = cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    _,frame = cap.read() #decode evry video frame
    canny_image = cv2.Canny(frame,50,50)
    cropped_image = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    averaged_lines = average_slope_intercept(frame,lines)
    line_image = display_lines(frame,averaged_lines)

    combo_image = cv2.addWeighted(frame,0.8,line_image,1,1)

    cv2.imshow('result',combo_image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()